In [102]:
from sqlalchemy import Table, Column, DateTime, Float, Integer, PrimaryKeyConstraint, MetaData, create_engine 

# Datenbank erstellen/verbinden
engine = create_engine('sqlite:///data.db')
metadata = MetaData()

# Tabelle definieren
aggregate = Table(
    'dae', metadata,
    Column('timestamp', DateTime, nullable=False),
    # Column('aggregate', Float, nullable=False),
    Column('washingmachine', Float, nullable=False),
    Column('fridge', Float, nullable=False),
    Column('microwave', Float, nullable=False),
    Column('kettle', Float, nullable=False),
    Column('user_id', Integer, nullable=False),

    PrimaryKeyConstraint('timestamp', 'user_id')
)

# Tabellen erstellen
metadata.create_all(engine)


In [98]:
query = f"""
            --DROP TABLE seq2point
            --DROP TABLE seq2seq
            --DROP TABLE aggregate
            DROP TABLE dae
            """
engine.execute(query)

In [5]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine('sqlite:///data.db')

query = f"""
            SELECT *
            FROM seq2point
            """
user_id = 1
# query = f"""
#         SELECT timestamp, aggregate
#         FROM aggregate a
#         join (select timestamp,washingmachine,microwave,kettle 
#                 from seq2point) sp 
#                 on a.timestamp = sp.timestamp
#         WHERE user_id = {user_id}
#         """

df = pd.read_sql_query(query, engine)
df

,timestamp,washingmachine,fridge,microwave,kettle,user_id


In [19]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker

# Erstellen der Verbindung zur SQLite-Datenbank
engine = create_engine('sqlite:///data.db')

# Erstellen der Session
Session = sessionmaker(bind=engine)
session = Session()

# Metadaten-Objekt erstellen
metadata = MetaData()

# Tabellennamen laden
table_names = engine.table_names()

# Daten aus jeder Tabelle abrufen
for table_name in table_names:
    table = Table(table_name, metadata, autoload_with=engine)
    query = table.select()
    results = session.execute(query).fetchall()
    print(f"Data from table {table_name}:")
    for result in results:
        # print(result)
        pass

# Schließen der Session
session.close()

C:\Users\stefa\AppData\Local\Temp\ipykernel_10484\1861582393.py:15: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  table_names = engine.table_names()


Data from table aggregate:
Data from table dae:
Data from table powerconsumption:
Data from table seq2point:
Data from table seq2seq:


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
user_id = 1
appliance = "microwave"
engine = create_engine('sqlite:///data.db')

def update_dataDB(df,engine,user_id):
    # vorhandene Daten auslesen
    df_existing = pd.read_sql('powerconsumption', con=engine)

    # angemeldeter Benutzer
    df_existing_update = df_existing[df_existing['user_id'] == user_id]
    # andere Benutzer
    df_existing_other = df_existing[df_existing['user_id'] != user_id]

    # Daten vom angemeldeten Benutzer zusammenführen -> neuen Daten haben Vorrang
    df_combined = pd.concat([df_existing_update, df])#.drop_duplicates(subset='timestamp', keep='last')

    # andere Daten wieder hinzufügen
    df_combined = df_combined.sort_values('timestamp').drop_duplicates(subset=['timestamp', 'user_id'], keep='last')
    df_combined = pd.concat([df_combined, df_existing_other]).sort_values('timestamp')

    # Daten in der DB aktualisieren
    df_combined.to_sql('powerconsumption', con=engine, if_exists='replace', index=True)

def predict_seq2seq(read_csv, write_prediction,db_engine,user_id):

    data = pd.read_csv(read_csv,header=None).rename(columns={0:'timestamp',1:'aggregate'})
    data['timestamp'] = pd.to_datetime(data['timestamp'])
    data = data.set_index('timestamp')

    aggregate = data['aggregate']
    mean_agg, std_agg, aggregate = normalise(aggregate)

    aggregate = np.pad(aggregate, (WINDOW_SIZE//2, WINDOW_SIZE//2 +1))
    aggregate = np.array([aggregate[i:i+WINDOW_SIZE] for i in range(len(aggregate)-WINDOW_SIZE)])
    aggregate = np.expand_dims(aggregate, axis=-1)


    model = return_seq2point()
    for appliance in SUPPORTED_APPLIANCES.keys():
        WEIGHTS_PATH = os.path.join(BASEDIR,"weights",SUPPORTED_APPLIANCES[appliance]['seq2point_weights'])
        model.load_weights(WEIGHTS_PATH)
        predict = model.predict(aggregate)

        data[appliance] = np.squeeze(predict)
    
    data['user_id'] = user_id


    data.to_csv(write_prediction)
    update_dataDB(data,db_engine,user_id)

    return True

In [38]:
import pandas as pd

#     Column('washingmachine', Float, nullable=False),
#     Column('fridge', Float, nullable=False),
#     Column('microwave', Float, nullable=False),
#     Column('kettle', Float, nullable=False),

query = f"""
            SELECT *
            FROM aggregate
            """
user_id = 1
# query = f"""
#         SELECT timestamp, aggregate
#         FROM aggregate a
#         join (select timestamp,washingmachine,microwave,kettle 
#                 from seq2point) sp 
#                 on a.timestamp = sp.timestamp
#         WHERE user_id = {user_id}
#         """


df = pd.read_sql_query(query, engine)
df

# df['timestamp'] = pd.to_datetime(df['timestamp'])
# df = df.set_index('timestamp')
# hourly_sum = df[appliance].resample('H').sum()
# hourly_sum.index = hourly_sum.index.strftime('%Y-%m-%d %H:%M')

# labels = hourly_sum.index.to_list()
# values = hourly_sum.values.tolist()
# mean = round(hourly_sum.values.mean()/1000,2)
# bills = round(hourly_sum.values.sum()*0.008/12,2)

,timestamp,aggregate,user_id
0,2023-06-01 00:00:00.000000,94.66300,4
1,2023-06-01 00:00:10.000000,95.13700,4
2,2023-06-01 00:00:20.000000,94.66800,4
3,2023-06-01 00:00:30.000000,94.78000,4
4,2023-06-01 00:00:40.000000,95.22500,4
...,...,...,...
509755,2023-04-29 23:59:10.000000,270.15100,4
509756,2023-04-29 23:59:20.000000,270.21500,4
509757,2023-04-29 23:59:30.000000,270.74298,4
509758,2023-04-29 23:59:40.000000,271.50900,4


In [20]:
from datetime import datetime,timedelta
df = pd.read_sql_query(query, engine)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.set_index('timestamp')
df['kWh'] = df['aggregate'] * (1/1000) * (1/360)
data = df['kWh']
data

timestamp
2023-06-01 00:00:00    0.000263
2023-06-01 00:00:10    0.000264
2023-06-01 00:00:20    0.000263
2023-06-01 00:00:30    0.000263
2023-06-01 00:00:40    0.000265
                         ...   
2023-04-29 23:59:10    0.000750
2023-04-29 23:59:20    0.000751
2023-04-29 23:59:30    0.000752
2023-04-29 23:59:40    0.000754
2023-04-29 23:59:50    0.000753
Name: kWh, Length: 509760, dtype: float64

In [21]:
six_months_ago = datetime.now() - timedelta(days=180)
monthly = data[data.index >= six_months_ago]
monthly = monthly.resample('M').sum().sort_index(ascending=False)
monthly.index = monthly.index.strftime('%b. %Y')
monthly.index.to_list()

['Jun. 2023', 'May. 2023', 'Apr. 2023']

In [23]:
[round(i,2) for i in monthly.values.tolist()]

[101.67, 213.01, 125.61]

app.db auslesen


-----------------

In [28]:
def get_energy_data(appliance,user_id,model_name):

    # predict(model_name,user_id)

    query = f"""
        SELECT timestamp,{appliance}
        FROM {model_name} 
        WHERE user_id = {user_id}
        """

    df = pd.read_sql_query(query, engine)

    if len(df) > 2:
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df = df.set_index('timestamp')
        data = df[appliance]
        hourly_sum = data.resample('H').sum()
        hourly_sum.index = hourly_sum.index.strftime('%Y-%m-%d %H:%M')

        monthly_sum = data.resample('H').sum().resample('M').mean()  # Summieren auf Monatsbasis
        monthly_sum.index = monthly_sum.index.strftime('%Y-%m')

        yearly_sum = data.resample('H').sum().resample('Y').mean()
        yearly_sum.index = yearly_sum.index.strftime('%Y')

        labels = hourly_sum.index.to_list()
        values = hourly_sum.values.tolist()
        mean = round(monthly_sum.values.mean()/1000, 2)
        bills = round(yearly_sum.values.mean()*0.008, 2)

    else:
        labels=["no data"]
        values=[0]
        mean = 0
        bills = 0

    return labels, values, mean, bills

labels, values, mean, bills = get_energy_data('washingmachine',4,'seq2point')
values

[2706.0414304733276,
 2337.2217540740967,
 874.9559720233083,
 278.80027355998755,
 1342.1565590202808,
 544.1448454856873,
 1.4298813566565514,
 72.37630828469992,
 890.5296133607626,
 1821.118899822235,
 1531.641995280981,
 517.5099785923958,
 1664.6109152138233,
 808.9962653964758,
 959.869651272893,
 256.94102693349123,
 461.87112913280725,
 1035.3400824666023,
 2240.928034596145,
 1988.0233632326126,
 2678.941358566284,
 2655.1296248435974,
 2554.778379201889,
 661.269438534975,
 671.8433289304376,
 1038.6895137280226,
 54.83269722759724,
 16.924303986132145,
 17.83835558593273,
 198.6769918128848,
 12.87958525121212,
 24.04048401117325,
 11.584650486707687,
 1366.151534512639,
 1073.6821835041046,
 2510.2161684036255,
 2476.8037037849426,
 2520.2640886306763,
 1963.39999127388,
 1738.777332663536,
 2347.254555940628,
 2354.4191126823425,
 1951.2583734989166,
 1019.2445087879896,
 2551.2313952445984,
 2672.6566615104675,
 1063.4843915253878,
 46.77775903791189,
 20.15266877412796,

In [37]:
def get_energy_data2(appliance,user_id,model_name):

    # predict(model_name,user_id)

    query = f"""
        SELECT timestamp,{appliance}
        FROM {model_name} 
        WHERE user_id = {user_id}
        """

    df = pd.read_sql_query(query, engine)

    if len(df) > 2:
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df = df.set_index('timestamp').sort_index(ascending=False)
        df['Wh'] = df[appliance] * (10/3600) # 10 = SampleRate
        data = df['Wh']

        hourly_sum = data.resample('H').sum().sort_index(ascending=True)
        hourly_sum.index = hourly_sum.index.strftime('%Y-%m-%d %H:%M')
        labels = hourly_sum.index.to_list()
        values = hourly_sum.values.tolist()


        monthly = data.resample('M').sum().sort_index(ascending=False)
        monthly.index = monthly.index.strftime('%b. %Y')

        mean = round(monthly.values.mean()/1000, 2)
        bills = round(mean*12*0.3, 2)

    else:
        labels=["no data"]
        values=[0]
        mean = 0
        bills = 0

    return labels, values, mean, bills

labels, values, mean, bills = get_energy_data2('washingmachine',4,'seq2point')
values

[7.5167817513148,
 6.492282650205825,
 2.4304332556203008,
 0.7744452043332988,
 3.7282126639452247,
 1.5115134596824646,
 0.003971892657379309,
 0.2010453007908331,
 2.4736933704465627,
 5.05866361061732,
 4.254561098002726,
 1.4375277183122106,
 4.623919208927287,
 2.247211848323544,
 2.6663045868691473,
 0.7137250748152535,
 1.2829753587022423,
 2.87594467351834,
 6.224800096100403,
 5.522287120090591,
 7.441503773795234,
 7.375360069009993,
 7.096606608894136,
 1.836859551486042,
 1.8662314692512156,
 2.8852486492445073,
 0.15231304785443678,
 0.047011955517033736,
 0.049550987738702035,
 0.551880532813569,
 0.03577662569781145,
 0.06677912225325902,
 0.032179584685299134,
 3.79486537364622,
 2.9824505097336242,
 6.972822690010071,
 6.8800102882915075,
 7.000733579529657,
 5.453888864649667,
 4.829937035176489,
 6.520151544279522,
 6.540053090784285,
 5.420162148608102,
 2.8312347466333048,
 7.08675387567944,
 7.424046281973522,
 2.954123309792744,
 0.12993821954975526,
 0.05597963

------------------

In [64]:
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker

# Erstellen der Verbindung zur SQLite-Datenbank
engine = create_engine('sqlite:///app.db')

# Erstellen der Session
Session = sessionmaker(bind=engine)
session = Session()

# Metadaten-Objekt erstellen
metadata = MetaData()

# Tabellennamen laden
table_names = engine.table_names()

# Daten aus jeder Tabelle abrufen
for table_name in table_names:
    table = Table(table_name, metadata, autoload_with=engine)
    query = table.select()
    results = session.execute(query).fetchall()
    print(f"Data from table {table_name}:")
    for result in results:
        print(result)

# Schließen der Session
session.close()


Data from table ab_permission:
(1, 'can_this_form_post')
(2, 'can_this_form_get')
(3, 'can_list')
(4, 'can_show')
(5, 'can_add')
(6, 'can_download')
(7, 'can_edit')
(8, 'can_delete')
(9, 'can_userinfo')
(10, 'resetmypassword')
(11, 'resetpasswords')
(12, 'userinfoedit')
(13, 'menu_access')
(14, 'copyrole')
(15, 'can_chart')
(16, 'can_get')
(17, 'can_method1')
(18, 'can_method2')
(19, 'can_home')
(20, 'can_logout')
(21, 'can_appliance')
(22, 'can_history')
(23, 'can_dashboard')
Data from table ab_permission_view:
(1, 1, 5)
(2, 2, 5)
(3, 1, 6)
(4, 2, 6)
(5, 1, 7)
(6, 2, 7)
(7, 3, 9)
(8, 4, 9)
(9, 5, 9)
(10, 6, 9)
(11, 7, 9)
(12, 8, 9)
(13, 9, 9)
(14, 10, 9)
(15, 11, 9)
(16, 12, 9)
(17, 13, 10)
(18, 13, 11)
(19, 3, 12)
(20, 4, 12)
(21, 5, 12)
(22, 6, 12)
(23, 7, 12)
(24, 8, 12)
(25, 14, 12)
(26, 13, 13)
(27, 15, 14)
(28, 13, 15)
(29, 3, 16)
(30, 13, 17)
(31, 3, 18)
(32, 13, 19)
(33, 3, 20)
(34, 13, 21)
(35, 16, 22)
(36, 13, 24)
(37, 13, 25)
(38, 13, 26)
(39, 17, 23)
(40, 18, 23)
(41, 19, 

C:\Users\stefa\AppData\Local\Temp\ipykernel_10484\2455109047.py:15: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  table_names = engine.table_names()


In [87]:
id = 4
query = f'''select first_name, last_name from ab_user where id = {id}'''
name = engine.execute(query).fetchall()[0]
f'{name[0]} {name[1]}'

'Stefan Dimnik'